# DS-SF-30 | Codealong 16: Trees

In [14]:
import os

import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 10)

import math

## Part A - The 2008 Democratic Primaries

(dataset adapted from http://www.stat.ucla.edu/~cocteau/primaries.csv)

In [32]:
df = pd.read_csv(os.path.join('..', 'datasets', 'dataset-16-2008-democrat-primaries.csv'))

In [33]:
df.columns

Index([u'fips', u'county_name', u'state_postal', u'region', u'election_date',
       u'racetype', u'tvotes', u'clinton', u'obama', u'edwards', u'margin',
       u'winner', u'POP05_SQMI', u'popUnder30_00', u'pop65up_00',
       u'presVote04', u'kerry04', u'Bush04', u'pres04margin', u'pres04winner',
       u'pop06', u'pop00', u'hisp06', u'white06', u'black06', u'indian06',
       u'asian06', u'hawaii06', u'mixed06', u'pct_less_30k', u'pct_more_100k',
       u'pct_hs_grad', u'pct_labor_force', u'pct_homeowner', u'unempFeb07',
       u'unempFeb08', u'unempChg', u'pctUnins00', u'subForPctHomes',
       u'poverty05', u'median_hhi05', u'Catholic', u'So.Bapt.Conv',
       u'Un.Methodist', u'E.L.C.A.', u'Construction', u'Manufacturing',
       u'FinancialActivities', u'GoodsProducing', u'ServiceProviding'],
      dtype='object')

In [34]:
df.winner.value_counts()
#shows only 2 possible values

clinton    1210
obama      1031
Name: winner, dtype: int64

In [35]:
# TODO classifications variable 
#ned to define class label
df['c'] = (df.winner =='obama')

In [36]:
df.c.value_counts()

False    1230
True     1031
Name: c, dtype: int64

### First cut: Is a county more than 20% black?

In [37]:
#first need to get ratio values of what % of each county is black
df['pct_black06'] = df.black06/df.pop06

In [38]:
df['pct_black06'] 

0       0.172109
1       0.096363
2       0.462710
3       0.219021
4       0.015451
          ...   
2256    0.008952
2257    0.002022
2258    0.000989
2259    0.001023
2260    0.001331
Name: pct_black06, dtype: float64

#### First cut/right node

In [39]:
# TODO
current_node = df
left_child_df = current_node[df.pct_black06 <= .2]
#right_child = current_node(df.pct_black06 > .2) #but it's smarter to do this instead...
right_child_df = current_node.drop(left_child_df.index)

In [40]:
def obama_vs_clinton(df):
    obama = (df.c == 1).sum()
    clinton = (df.c == 0).sum()
    if obama > clinton:
        print 'Obama wins these counties {} to {}.'.format(obama, clinton)
    elif clinton > obama:
        print 'Clinton wins these counties {} to {}.'.format(clinton, obama)
    else:
        print 'Obama and Clinton tie in these counties {} {}.'.format(obama, clinton)

In [41]:
obama_vs_clinton(right_child_df)

Obama wins these counties 381 to 70.


### Second cut: Is high school graduation rate higher than 78%?

In [42]:
# TODO
current_node = left_child_df
left_child_df = current_node[df.pct_hs_grad <= .78]
right_child_df = current_node.drop(left_child_df.index)

C:\Users\tcaballero\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()


In [43]:
obama_vs_clinton(left_child_df)
#should be 714 to 93

Clinton wins these counties 714 to 93.


### Third cut: Is high school graduation rate higher than 87%?

In [44]:
# TODO
current_node = right_child_df
left_child_df = current_node[df.pct_hs_grad <= .87]
#right_child = current_node(df.pct_black06 > .2) #but it's smarter to do this instead...
right_child_df = current_node.drop(left_child_df.index)

C:\Users\tcaballero\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()


In [45]:
obama_vs_clinton(right_child_df)

Obama wins these counties 177 to 36.


## Part B - Building the 2008 Democratic Primaries Decision Tree by Hand

In [46]:
class Node:

    @staticmethod
    def root(root_df):
        cs = sorted(set(root_df.c))
        return Node(cs, root_df)

    def decision(self, left_filter):
        # Collect the observations for which the decision split is true and
        # create the corresponding left node

        left_filter = left_filter(self.df)
        left_df = self.df[left_filter]
        self.left = Node(self.cs, left_df)

        # Same thing on the right side but for the observations that don't
        # satisfy the decision split (the "else")

        right_df = self.df.drop(left_df.index)
        self.right = Node(self.cs, right_df)

        # The entropy after the decision split is the weighted average of the
        # children entropy

        self.after = (self.left.samples * self.left.before
                      + self.right.samples * self.right.before) / self.samples

        # The information gain corresponds to the entropy lost between the
        # parent node (this node and the "before") and its child (the "after")

        self.information_gain = self.before - self.after

        return self

    def __init__(self, cs, df):
        self.cs = cs
        self.df = df

        # Counts of the remaining observations in the subspace per classes
        self.counts = [(self.df.c == c).sum() for c in self.cs]

        # Number of observations in the subspace
        self.samples = sum(self.counts)

        # For empty subspaces, probabilties and entropy are set to zero
        if self.samples == 0:
            self.probabilities = [.0 for count in self.counts]
            self.before = .0
        else:
            self.probabilities = [1. * count / self.samples for count in self.counts]
            self.before = - sum(map(lambda p: p * math.log(p, 2),
                                    filter(lambda p : p > .0, self.probabilities)))

    def status(self):
        print 'classes                       =', self.cs
        print 'before:'
        print "\tparent:"
        print "\t\tsamples       =", self.samples
        print "\t\tcounts        =", self.counts
        print "\t\tprobabilities =", self.probabilities
        print "\t\tentropy       =", self.before
        print 'after:'
        print "\tleft child:"
        print "\t\tsamples       =", self.left.samples
        print "\t\tcounts        =", self.left.counts
        print "\t\tprobabilities =", self.left.probabilities
        print "\t\tentropy       =", self.left.before
        print "\tright child:"
        print "\t\tsamples       =", self.right.samples
        print "\t\tcounts        =", self.right.counts
        print "\t\tprobabilities =", self.right.probabilities
        print "\t\tentropy       =", self.right.before
        print
        print 'before entropy                =', self.before
        print 'after entropy                 =', self.after
        print 'information gain              =', self.information_gain

In [47]:
df.c = df.winner

### First cut

In [48]:
# TODO set node to root
node = Node.root(df)

#### Candidate #1: Is a county more than 20% black?

In [51]:
# split1
node.decision(lambda df: df.pct_black06 <= .2).status()


classes                       = [nan, 'clinton', 'obama']
before:
	parent:
		samples       = 2241
		counts        = [0, 1210, 1031]
		probabilities = [0.0, 0.53993752788933513, 0.46006247211066487]
		entropy       = 0.995392878882
after:
	left child:
		samples       = 1791
		counts        = [0, 1141, 650]
		probabilities = [0.0, 0.63707426018983804, 0.36292573981016191]
		entropy       = 0.945085004347
	right child:
		samples       = 450
		counts        = [0, 69, 381]
		probabilities = [0.0, 0.15333333333333332, 0.84666666666666668]
		entropy       = 0.6181194891

before entropy                = 0.995392878882
after entropy                 = 0.879429278394
information gain              = 0.115963600488


#### Candidate #2: Is high school graduation rate higher than 78%?

In [52]:
# TODO
node.decision(lambda df: df.pct_hs_grad <= .78).status()
#info gain = .04 which is worse than .1159 above, so above is better

classes                       = [nan, 'clinton', 'obama']
before:
	parent:
		samples       = 2241
		counts        = [0, 1210, 1031]
		probabilities = [0.0, 0.53993752788933513, 0.46006247211066487]
		entropy       = 0.995392878882
after:
	left child:
		samples       = 1167
		counts        = [0, 774, 393]
		probabilities = [0.0, 0.66323907455012854, 0.33676092544987146]
		entropy       = 0.92168535501
	right child:
		samples       = 1074
		counts        = [0, 436, 638]
		probabilities = [0.0, 0.4059590316573557, 0.5940409683426443]
		entropy       = 0.974329848577

before entropy                = 0.995392878882
after entropy                 = 0.946915246171
information gain              = 0.0484776327111


#### Candidate #3: Is high school graduation rate higher than 87%?

In [55]:
# TODO
node.decision(lambda df: df.pct_hs_grad <= .87).status()

AttributeError: 'NoneType' object has no attribute 'decision'

### Second cut

In [53]:
# TODO
node = node.decision(lambda df: df.pct_black06 <= .2).status()

classes                       = [nan, 'clinton', 'obama']
before:
	parent:
		samples       = 2241
		counts        = [0, 1210, 1031]
		probabilities = [0.0, 0.53993752788933513, 0.46006247211066487]
		entropy       = 0.995392878882
after:
	left child:
		samples       = 1791
		counts        = [0, 1141, 650]
		probabilities = [0.0, 0.63707426018983804, 0.36292573981016191]
		entropy       = 0.945085004347
	right child:
		samples       = 450
		counts        = [0, 69, 381]
		probabilities = [0.0, 0.15333333333333332, 0.84666666666666668]
		entropy       = 0.6181194891

before entropy                = 0.995392878882
after entropy                 = 0.879429278394
information gain              = 0.115963600488


### Third cut

In [ ]:
# TODO